### Using a jupyter notebook to be able to display the Arabic text in it's correct format (vs termnial)

Note that this notebook will be deleted and the working (cleaner) code will be transfered to insert_data.py, 
once we have validated that the script works fine In Shaa Allah

In [1]:
# Import relevent libraries 
import os
import psycopg2
import pandas as pd
from psycopg2 import Error

In [2]:
# Read in the data file that we need to insert
df = pd.read_excel("/Users/tahir/Desktop/Github/documents/roots/Data/Cleaned_Root_letters.xlsx")

In [3]:
df.head()

,ID,ARABIC,Transliteration,Root_Letters
0,1:1,بِسْمِ,bis'mi,س م و
1,1:1,اللَّهِ,l-lahi,ا ل ه
2,1:1,الرَّحْمَنِ,l-raḥmāni,ر ح م
3,1:1,الرَّحِيمِ,l-raḥīmi,ر ح م
4,1:2,الْحَمْدُ,al-ḥamdu,ح م د


In [4]:
# Prep data for the RootWords Table 
# Get only the unique root words
lst = list(df['Root_Letters'])
lst = set(lst)

# Make it into a dataframe, and give it an index / ID column 
df_roots = pd.DataFrame(lst, columns=['Root_Words'])
df_roots = df_roots.reset_index()

In [5]:
df_roots.head()

,index,Root_Words
0,0,ك ث ر
1,1,م ر ر
2,2,ب ي د
3,3,س ل ط
4,4,ض ي ع


In [142]:
df_roots["Root_Words"] = df_roots["Root_Words"].astype(str)
#total_rows['ColumnID'] = total_rows['ColumnID'].astype(str)

In [143]:
lst = []
for word in df_roots["Root_Words"]:
    lst.append(str(word))

In [144]:
# df_roots["Root_Words"] = lst
df_roots["Root_Words"] = df_roots["Root_Words"].values.astype(str)

In [145]:
df_roots.dtypes

index          int64
Root_Words    object
dtype: object

In [6]:
data = []
for row in df_roots.index:
    arabic_text = df_roots.loc[row,"Root_Words"]
    unique_id = row
    data.append((row, arabic_text))

In [7]:
data

[(0, 'ك ث ر '),
 (1, 'م ر ر '),
 (2, 'ب ي د '),
 (3, 'س ل ط '),
 (4, 'ض ي ع '),
 (5, 'ر ح ق '),
 (6, 'ط ر ق '),
 (7, 'ز ل ف '),
 (8, 'و ل د '),
 (9, 'م ه م ا '),
 (10, 'ا ن ن '),
 (11, 'ج ز ي '),
 (12, 'س ن د س '),
 (13, 'ع ض د '),
 (14, 'و ص ي '),
 (15, 'ن ف ح '),
 (16, 'ص ل ي '),
 (17, 'ع ر م '),
 (18, 'ك ف ف '),
 (19, 'ح ك م '),
 (20, 'ا ن '),
 (21, 'م ز ج '),
 (22, 'خ ط ط '),
 (23, 'ا ف ك '),
 (24, 'و ر د '),
 (25, 'ز م ل '),
 (26, 'ر ا ي '),
 (27, 'ل و ت '),
 (28, 'ط ل ل '),
 (29, 'و ض ن '),
 (30, 'ع ق د '),
 (31, 'ش خ ص '),
 (32, 'ا ن و '),
 (33, 'ن ه ي '),
 (34, 'و ق ر '),
 (35, 'ك ر ب '),
 (36, 'ث ب ط '),
 (37, 'ج ل ي '),
 (38, 'ش ي ب '),
 (39, 'ش ر ع '),
 (40, 'ر ه ق '),
 (41, 'م ر ج '),
 (42, 'ص ع ق '),
 (43, 'ن و م '),
 (44, 'ش ف ع '),
 (45, 'ص ع د '),
 (46, 'ص ر ر '),
 (47, 'م ش ي '),
 (48, 'ب ط ش '),
 (49, 'ر ق د '),
 (50, 'ك م ل '),
 (51, 'ن ص ب '),
 (52, 'د ر ر '),
 (53, 'ب ت ك '),
 (54, 'ع س ي '),
 (55, 'م ل ح '),
 (56, 'ف س ق '),
 (57, 'ع و ج '),
 (58, 'ر ج ل '),
 (59,

In [8]:
# Define parameters for the database as global variables
dbHost = '127.0.0.1'
dbPort = 5434
dbUser = 'qj'
dbPassword= 'Yatathakar123!'
dbName = 'quranJourney'

In [9]:
def insert_root_words(root_words_data):
    
    try:
        # connect to the PostgreSQL database
        connection = psycopg2.connect(user=dbUser,
                                  password=dbPassword,
                                  host=dbHost,
                                  port=dbPort,
                                  database=dbName)
        cursor = connection.cursor()

        postgres_insert_query = """ INSERT INTO RootWords (RootID, RootWord) VALUES (%s ,%s)"""
        counter = 0
        for row in root_words_data:
            record_to_insert = row
            cursor.execute(postgres_insert_query, record_to_insert)
            connection.commit()
            counter += cursor.rowcount
            print(count, "Record inserted successfully into rootWords table")

    except (Exception, psycopg2.Error) as error:
        print("Failed to insert record into rootwords table", error)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
    return counter  


In [14]:
import psycopg2
from psycopg2 import Error

try:
    # Connect to an existing database
    connection = psycopg2.connect(user="qj",
                                  password="Yatathakar123!",
                                  host="127.0.0.1",
                                  port="5434",
                                  database="quranJourney")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    #print("PostgreSQL server information")
    #print(connection.get_dsn_parameters(), "\n")
    
    # Executing a SQL query
    cursor.execute("SELECT version();")
    fetch_rows =  "SELECT * FROM RootWords;"
    cursor.execute(fetch_rows)
    # Fetch result
    record = cursor.fetchall()
    print("You are connected to - ", dbName, "\n")
    print(record)
    

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")



You are connected to -  quranJourney 

[(0, 'ر ت ع '), (1, 'ل ظ ي '), (2, 'ر ص د '), (3, 'ف ك ه '), (4, 'ه ل ك '), (5, 'ح ي ي  ا '), (6, 'ر ج ز '), (7, 'غ ض ض '), (8, 'ر د د '), (9, 'م ن ع '), (10, 'ق '), (11, 'د خ ل '), (12, 'ق ص د '), (13, 'ق ن ي '), (14, 'د س ر '), (15, 'ح ص ي '), (16, 'ن ف ث '), (17, 'ف ر ج '), (18, 'ك ع ب '), (19, 'ش ت ت '), (20, 'ح ط م '), (21, 'ل ف ي '), (22, 'ر س خ '), (23, 'ا س ف '), (24, 'ن ب ذ '), (25, 'ف ض ي '), (26, 'ر ب ح '), (27, 'ع ت ق '), (28, 'ا ي د '), (29, 'غ ل ب '), (30, 'ص ل ي '), (31, 'خ ر ق '), (32, 'ط و ف '), (33, 'س ر ج '), (34, 'غ ن م '), (35, 'ه ا ت ي '), (36, 'ط ح ا '), (37, 'ع ف ر '), (38, 'ن ج و '), (39, 'ش ي ب '), (40, 'ب ك م '), (41, 'ح ش ر '), (42, 'ط م ن '), (43, 'ب د ل '), (44, 'ح ر ك '), (45, 'ث ق ب '), (46, 'ص ي ف '), (47, 'ل ح د '), (48, 'ب ل س '), (49, 'ش ا ن '), (50, 'ع ص م '), (51, 'م ك ك '), (52, 'ح و ج '), (53, 'ه ا ك '), (54, 'ك ي ف '), (55, 'ج ز ي '), (56, 'ق ل م '), (57, 'ر ب ط '), (58, 'ح م ا '), (59, 'ع و ن '), (60, 'ا خ

In [16]:
print(len(record))
record

1799


[(0, 'ر ت ع '),
 (1, 'ل ظ ي '),
 (2, 'ر ص د '),
 (3, 'ف ك ه '),
 (4, 'ه ل ك '),
 (5, 'ح ي ي  ا '),
 (6, 'ر ج ز '),
 (7, 'غ ض ض '),
 (8, 'ر د د '),
 (9, 'م ن ع '),
 (10, 'ق '),
 (11, 'د خ ل '),
 (12, 'ق ص د '),
 (13, 'ق ن ي '),
 (14, 'د س ر '),
 (15, 'ح ص ي '),
 (16, 'ن ف ث '),
 (17, 'ف ر ج '),
 (18, 'ك ع ب '),
 (19, 'ش ت ت '),
 (20, 'ح ط م '),
 (21, 'ل ف ي '),
 (22, 'ر س خ '),
 (23, 'ا س ف '),
 (24, 'ن ب ذ '),
 (25, 'ف ض ي '),
 (26, 'ر ب ح '),
 (27, 'ع ت ق '),
 (28, 'ا ي د '),
 (29, 'غ ل ب '),
 (30, 'ص ل ي '),
 (31, 'خ ر ق '),
 (32, 'ط و ف '),
 (33, 'س ر ج '),
 (34, 'غ ن م '),
 (35, 'ه ا ت ي '),
 (36, 'ط ح ا '),
 (37, 'ع ف ر '),
 (38, 'ن ج و '),
 (39, 'ش ي ب '),
 (40, 'ب ك م '),
 (41, 'ح ش ر '),
 (42, 'ط م ن '),
 (43, 'ب د ل '),
 (44, 'ح ر ك '),
 (45, 'ث ق ب '),
 (46, 'ص ي ف '),
 (47, 'ل ح د '),
 (48, 'ب ل س '),
 (49, 'ش ا ن '),
 (50, 'ع ص م '),
 (51, 'م ك ك '),
 (52, 'ح و ج '),
 (53, 'ه ا ك '),
 (54, 'ك ي ف '),
 (55, 'ج ز ي '),
 (56, 'ق ل م '),
 (57, 'ر ب ط '),
 (58, 'ح م ا '),
 (59, 

In [136]:
insert_root_words(data)

Failed to insert record into rootwords table connection to server at "127.0.0.1", port 5434 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?



UnboundLocalError: local variable 'connection' referenced before assignment

In [ ]:
insert_stmt = (
  "INSERT INTO employees (emp_no, first_name, last_name, hire_date) "
  "VALUES (%s, %s, %s, %s)"
)
data = (2, 'Jane', 'Doe', datetime.date(2012, 3, 23))
cursor.execute(insert_stmt, data)

select_stmt = "SELECT * FROM employees WHERE emp_no = %(emp_no)s"
cursor.execute(select_stmt, { 'emp_no': 2 })

In [119]:
def insert_root_words(root_words_data):
    """ Insert multiple root words into the rootWords table  """
    
    sql = "INSERT INTO RootWords (RootID, RootWord) VALUES (%s, %s)"
    conn = None
    
    try:
        # connect to the PostgreSQL database
        connection = psycopg2.connect(user=dbUser,
                                  password=dbPassword,
                                  host=dbHost,
                                  port=dbPort,
                                  database=dbName)
        # create a new cursor
        cur = connection.cursor()
        # execute the INSERT statement
        cur.executemany(sql,root_words_data)
        # commit the changes to the database
        connection.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [120]:
if __name__ == '__main__':
    
    # insert multiple root words
    insert_root_words(data)

invalid input syntax for type integer: "ر ت ع "
LINE 1: INSERT INTO RootWords (RootID, RootWord) VALUES ('ر ت ع ', 0...
                                                         ^



In [105]:
# Connect to the DB using Psycopg2

# In Psycopg2 you need to create two objects 
# Connection object -> To connect to the db
# Cursor object -> Execute commands to fetch commands, delete rows, insert rows, etc. (Used to run your SQL commands on the db)

# Define parameters for the database 
dbHost = '127.0.0.1'
dbPort = 5434
dbUser = 'qj'
dbPassword= 'Yatathakar123!'
dbName = 'quranJourney'

try:
    connection = psycopg2.connect(user=dbUser,
                                  password=dbPassword,
                                  host=dbHost,
                                  port=dbPort,
                                  database=dbName)

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print("Connection Settings: ")
    print(connection.get_dsn_parameters(), "\n")
    
    # WRITE DATA INTO THE ROOT TABLE 
    for row in df_roots.index:
        #arabic_word = str(df_roots.loc[row,"Root_Words"])
        #print(arabic_word)
        #unique_id = row
        
        #SQLTEXT = "INSERT INTO RootWords(RootID, RootWord) VALUES ({}, {});".format(unique_id,arabic_word)
        
        # Execute the sql text to perform the insertion each time
        cursor.execute(SQLTEXT)
    
    #rowCount = cur.rowcount
    #print("number of inserted rows =", rowCount)

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        # Close the cursor
        cursor.close()
        # Close the connection to the db
        connection.close()
        #rowCount = cursor.rowcount
        #print("number of inserted rows =", rowCount)
        print("PostgreSQL connection is closed")



PostgreSQL server information
Connection Settings: 
{'user': 'qj', 'channel_binding': 'prefer', 'dbname': 'quranJourney', 'host': '127.0.0.1', 'port': '5434', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

ر ت ع 
Error while connecting to PostgreSQL syntax error at or near "ت"
LINE 1: INSERT INTO RootWords(RootID, RootWord) VALUES (0, ر ت ع );
                                                             ^

PostgreSQL connection is closed


In [89]:
CREATE TABLE IF NOT EXISTS RootWords (
    RootID INT NOT NULL,
    RootWord VARCHAR(225) NOT NULL,
    PRIMARY KEY (RootID)
);

CREATE TABLE IF NOT EXISTS ArabicWord (
    WordID INT NOT NULL,
    Word VARCHAR(255) NOT NULL,
    Transliteration VARCHAR(255) NOT NULL,
    RootID INT NOT NULL,
    PRIMARY KEY (WordID),
    FOREIGN KEY (RootID)
        REFERENCES RootWords(RootID)
        ON DELETE CASCADE
        ON UPDATE CASCADE
);

CREATE TABLE IF NOT EXISTS TextToWord  (
    AyahID INT NOT NULL,
    WordID INT NOT NULL,
    PRIMARY KEY (AyahID, WordID),
    FOREIGN KEY (AyahID)
        REFERENCES quran_text("index")
        ON DELETE CASCADE
        ON UPDATE CASCADE,
	FOREIGN KEY (WordID)
		REFERENCES ArabicWord(WordID)
		ON DELETE CASCADE
		ON UPDATE CASCADE
);



SyntaxError: invalid syntax (396337841.py, line 1)

In [ ]:
INSERT INTO "quran_text" ("index", "sura", "aya", "text") VALUES
(1, 1, 1, 'بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ'),
(2, 1, 2, 'الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ'),
(3, 1, 3, 'الرَّحْمَـٰنِ الرَّحِيمِ'),
(4, 1, 4, 'مَالِكِ يَوْمِ الدِّينِ'),
(5, 1, 5, 'إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ'),
(6, 1, 6, 'اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ'),
(7, 1, 7, 'صِرَاطَ الَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ الْمَغْضُوبِ عَلَيْهِمْ وَلَا الضَّالِّينَ');
